In [ ]:
# Imports
import pandas as pd
import numpy as np
from os import path

In [ ]:
# Definição dos dataframes iniciais
directory = "data"
food_df = pd.read_csv(path.join(directory, "food.csv"))
nutrient_df = pd.read_csv(path.join(directory, "nutrient.csv"))
food_nutrient_df = pd.read_csv(path.join(directory, "food_nutrient.csv"))

In [ ]:
# Cruzando os nutrientes

# Nutriente e o id dele
nutrient_ids = {          # Nome do nutriente na tabela
    "carbohydrate": 2039, # Carbohydrates
    "protein":1003,       # Protein
    "fat": 1004,          # Total lipid (fat)
    "water": 1051,        # Water
    "sugars": 1063,       # Sugars, Total
    "fiber": 1079         # Fiber, total dietary
}

# Ids das comidas e descricoes
foods = food_df[["fdc_id", "description"]].copy().set_index("fdc_id")

for n_name, n_id in nutrient_ids.items():
    # Pra cada nutriente, cria uma coluna vazia pra ele na tabela
    foods[n_name] = np.nan

    # Pega o fdc_id e a quantidade em todas as linhas referentes a esse nutriente 
    tmp = food_nutrient_df.query("nutrient_id == @n_id")[["fdc_id", "amount"]]

    # Pra cada linha, insere a quantidade na tabela
    for _, fdc_id, amount in tmp.itertuples():
        foods.at[fdc_id, n_name] = amount


# Tô vendo como fica se eu só dropar as linhas 100% vazias, pq se usar how="any", não sobra linha nenhuma no df
foods = foods.dropna(how="all")

# Tem que usar essa opção pro pandas imprimir a saída toda, mas eu nn queria definir isso pro arquivo inteiro
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(foods.head(100))